# Writing to DB

In [1]:
from __future__ import annotations

from typing import Dict, List

from pymongo import MongoClient
from pymongo.cursor import Cursor
from typing import List
from pymongo.cursor import Cursor
from typing import List, Optional


class MongoAPI:
    """
    inserts and reads with a connection to a mongodb database
    """

    def __init__(self, db: str):
        """
        initializes the MongoAPI object
        :param db: the name of the database to connect to
        """
        self.__client: MongoClient = MongoClient()[db]

    def batch_insert(self, collection: str, insert_me: List[Dict]) -> None:
        """
        inserts a list of documents into the specified collection of the desired mongo database.

        :param collection: the collection to insert to .
        :param insert_me: the document to be inserted into the database.
        :return: None
        """
        self.__client[collection].insert_many(insert_me)

    def read_from_db_agg(self, collection: str, query: List[Dict[str, any]]) -> Cursor:
        """
        passes a given aggregation query to the find method of pymongo
        :param collection: the collection to search
        :param query: the query to search the db for
        :return: the results of the query as a pymongo.cursor.Cursor
        """
        return self.__client[collection].aggregate(query)



import gc

import pandas as pd




class AssetQuery:
    """
    Handles and formats mongo queries for the client.
    Client interacts with this object

    Allows client the option to save the search results
    therefore not having to rerun queries all the time
    This is very helpful when working with jupyter notebooks

    Allows the api to not force the client to read query results into memory right away.
    Also allows for different frame works to be used instead of python like dask

    provides functionality for turning data into data frame.
    In the future this class can be extended to possibly cache the formatted 2d data if needed
    """

    def __init__(self, aggregation_query_results: Cursor, unique_identifiers: List[str], save: bool = False):
        """
        :param aggregation_query_results: the results of a aggregation query on the database
                MUST HAVE A CUSIP and DATE FIELD in the given query. 'cusip', 'date'
        :param unique_identifiers: the primary key/keys for the given aggregation_query_results
        :param save: weather or not to save the aggregation_query_results or exaust them when used
            if true the .df will use a copy of aggregation_query_results therefore not exhausting the object
        """
        self.__aggregation_query_results = aggregation_query_results
        self.__unique_identifiers = unique_identifiers
        self.__save = save
        self.__saved_df = None  # holds a copy of the query data in the dataframe if __save is True

    def set_save(self, save: bool = True) -> AssetQuery:
        """
        provides the functionality of saving the query and never exhausting the original copy
        :param save: bool weather or not to save the query so we dont have to keep re scanning the database
        :return: self, with the save indicator set to true
        """
        if not save:
            del self.__saved_df
            gc.collect()
        self.__save = save
        return self

    @property
    def df(self) -> pd.DataFrame:
        """
        Turns the aggregation query into a pandas dataframe
        :return: data frame
                columns are the parameters for the query index is a
                index is date, cusip
        """

        if self.__save:
            if self.__saved_df is None:
                self.__saved_df = pd.DataFrame(self.aggregation_query_results).set_index(self.__unique_identifiers)

            return self.__saved_df.copy()

        try:
            # catch a KeyError from setting indexes on a empty dataframe
            return pd.DataFrame(self.aggregation_query_results).set_index(self.__unique_identifiers)
        except KeyError:
            raise ValueError('The query contents have already been exhausted or the query has returned no results')

    @property
    def aggregation_query_results(self) -> Cursor:
        """
        If save is False then a mutable Cursor will be returned
        if Save is False then the real cursor will be returned
        :return: returns the raw cursor for the query
        """
        if self.__save:
            return self.__aggregation_query_results

        return self.__aggregation_query_results

    @property
    def unique_identifiers(self) -> List[str]:
        """
        :return: the identifiers for the query data
        """
        return self.__unique_identifiers
import pandas as pd


class ReadDB:
    """
    This class provides functionality to read from the given equity database
    """

    def __init__(self, api: MongoAPI):
        """
        user is specifying the connection to the mongo database
        :param api: the connection to use for inserting data
        """
        self.__api = api

    def get_asset_data(self, assets: List[str], ts_fields: Optional[List[str]] = None,
                       static_fields: Optional[List[str]] = None, start: pd.Timestamp = None,
                       end: pd.Timestamp = None, search_by: str = 'cusip') -> AssetQuery:
        """
        queries the database according to the inputs provided by the user.
        returns a AssetQuery object which the user can use to derive their preferred form of output\

        :param assets: the assets to search the database by
        :param ts_fields: the time series data fields we want to search for in our query
        :param static_fields: the static data fields we want to include in out query
        :param start: the start time frame for the time series data
        :param end: the end time for the time series data
        :param search_by: the field name of the identifier we passed in "assets"
        :return: a AssetQuery object representing the object
        """

        if (not ts_fields) and (not static_fields):
            raise ValueError('Either ts_fields or static_fields must be passed')

        if (bool(start) + bool(end) != 2) and ts_fields:
            raise ValueError('Both start and end must be specified if querying time series data')

        # this always needs to be made no matter if there is no static wanted
        static_projection = {field: 1 for field in (static_fields if static_fields else [])}
        static_projection['_id'] = 0

        if ts_fields:
            # making the timeseries projection dict
            timeseries_projection = {field: '$timeseries.' + field for field in ts_fields}
            timeseries_projection['date'] = '$timeseries.datadate'

            aggregation_query = [
                {'$match': {search_by: {'$in': assets}}},
                {'$unwind': "$timeseries"},
                {'$match': {'timeseries.datadate': {'$gte': start, '$lt': end}}},
                {'$project': {**static_projection, **timeseries_projection}}
            ]
            primary_key = ['date', search_by]

        else:
            aggregation_query = [{'$match': {search_by: {'$in': assets}}},
                                 {'$project': static_projection}]
            primary_key = [search_by]

        query_results = self.__api.read_from_db_agg('compustat', aggregation_query)
        return AssetQuery(aggregation_query_results=query_results, unique_identifiers=primary_key)

import gc

import pandas as pd

from typing import List, Optional
from tqdm import tqdm

class InsertIntoDB:
    """
    a class to insert compustat finical information a mongo db
    """

    def __init__(self, api: MongoAPI):
        """
        user is specifying the connection to the mongo database
        :param api: the connection to use for inserting data
        """
        self.api = api

    def format_and_insert(self, data: pd.DataFrame, timeseries_cols: Optional[List[str]] = None,
                          static_cols: Optional[List[str]] = None) -> None:
        """
        Takes compustat 2-d data, formats it into a document structure
        then inserts the document into a mongo database
        MUTATES the given df


        :param data: the data to be entered into the mongo database
            MUST CONTAIN columns: 'datadate', 'cusip'
            Must have range index on dataframe
        :param timeseries_cols: the columns in the passed data that contain timeseries data
            if nothing is passed then the default values will be used
        :param static_cols: the columns in the passed data that contain static data
            if nothing is passed then the default values will be used
        :return: None

        Insert format:

        {ticker   : "AAPL",
         CUSIP    : "12346",
         static2  : "blah blah",
         timeseries: [
            {date: date_object(2010-01-02),
             close: 120},
            {date: date_object(2010-01-03),
            close: 121}
            ]
         },
        """
        # seeing weather or not to use default, this is cleaner then setting default param
        if not static_cols:
            static_cols = ['cusip', 'tic', 'cik', 'conm', 'exchg', 'add1', 'spcindcd', 'weburl']
        if not timeseries_cols:
            timeseries_cols = ["div", "ajexdi", "cshoc", "cshtrd", "eps", "prccd", "prchd", "prcld", "prcod"]

        print('Parsing dates')
        data['datadate'] = pd.to_datetime(data['datadate'], format='%Y%m%d')

        print('adjusting indexes of passed frame')
        cusip_list = data['cusip'].unique()
        data.set_index(['cusip', 'datadate'], inplace=True)

        # giving user some info
        print(f'Inserting {len(data)} rows of data for {len(cusip_list)} unique assets')

        documents_to_be_inserted = []
        for ticker in tqdm(cusip_list):  # will print progress bar
            # formatting the document data for a single asset at a time
            data_tick = data.xs(ticker)
            static_df = data_tick.iloc[0].to_frame().reindex(static_cols)

            ticker_dict = list(static_df.to_dict().values())[0]
            ticker_dict['cusip'] = ticker
            ticker_dict['timeseries'] = list(data_tick[timeseries_cols].reset_index().to_dict('index').values())

            documents_to_be_inserted.append(ticker_dict)

            # batch inserting documents into the data base in batches of 20
            if len(documents_to_be_inserted) == 100:
                # batch inserting to the db
                self.api.batch_insert('compustat', documents_to_be_inserted)
                del documents_to_be_inserted
                documents_to_be_inserted = []
                gc.collect()

        # doing last check to ensure there is nothing left over in the documents_to_be_inserted
        if documents_to_be_inserted:
            # batch inserting to the db
            self.api.batch_insert('compustat', documents_to_be_inserted)


In [2]:
# reading our file
data = pd.read_csv("DS4300_data_2010.csv")

C:\Users\payto\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,4,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Lets see what we are dealing with
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51242297 entries, 0 to 51242296
Data columns (total 20 columns):
 #   Column    Dtype  
---  ------    -----  
 0   gvkey     int64  
 1   iid       object 
 2   datadate  int64  
 3   tic       object 
 4   cusip     object 
 5   conm      object 
 6   div       float64
 7   ajexdi    float64
 8   cshoc     float64
 9   cshtrd    float64
 10  eps       float64
 11  prccd     float64
 12  prchd     float64
 13  prcld     float64
 14  prcod     float64
 15  exchg     float64
 16  cik       float64
 17  add1      object 
 18  spcindcd  float64
 19  weburl    object 
dtypes: float64(12), int64(2), object(6)
memory usage: 7.6+ GB


In [4]:
# turning the data into strings and ints
# nupy objects are not accepted by Mongo
string = ['cusip', 'tic', 'cik', 'conm', 'exchg', 'add1', 'spcindcd', 'weburl']
integer  = ["div", "ajexdi", "cshoc", "cshtrd", "eps", "prccd", "prchd", "prcld", "prcod"]

for col in string:
    data[col] = data[col].astype(str)

for col in integer:
    data[col] = data[col].astype(float)

In [5]:
# connecting to the mongo db
api = MongoAPI('DS4300')

# making the object which gives us write acess
insert_object = InsertIntoDB(api)

In [6]:
from datetime import datetime

In [7]:
# formatting the table into a document
# inserting that document into a mongo db collection called "compustat"
start = datetime.now()
print(start)
insert_object.format_and_insert(
    data,
    static_cols= ['cusip', 'tic', 'cik', 'conm', 'exchg', 'add1', 'spcindcd', 'weburl'],
    timeseries_cols=['div', 'ajexdi', 'cshoc', 'cshtrd', 'eps', 'prccd', 'prchd', 'prcld', 'prcod']
)
end = datetime.now()
print(end)
print("Upload Time: ", (end - start))

2021-04-19 15:51:08.526840
Parsing dates
adjusting indexes of passed frame


  0%|          | 0/33067 [00:00<?, ?it/s]

Inserting 51242297 rows of data for 33067 unique assets


100%|██████████| 33067/33067 [1:12:12<00:00,  7.63it/s]


2021-04-19 17:04:18.436149
Upload Time:  1:13:09.909309
